# Bloon Bot

## Models to research

* Genetic Algorithm

* Proximal Policy Optimization

## Goal: Maximize rounds survived

**Rewards:**

* Passing rounds

* Starting rounds with leftover money

* Starting rounds with less towers placed

* Higher pop count

**Penalties:**

* Losing lives

* Rounds passed until game over (pentalty decreases as rounds survived increases)

In [2]:
import pytesseract
import re
import pyautogui
import pydirectinput
import time
from PIL import ImageGrab, Image, ImageDraw
import cv2
import numpy as np
import keyboard
import matplotlib.pyplot as plt
import pandas as pd

import bloon_functions as bfn

In [25]:
def screen_cap():
    pyautogui.hotkey('alt', 'prtscr')
    img = ImageGrab.grabclipboard()
    img.save('temp.png')
    return(cv2.imread('temp.png'))


def round_state():
    time.sleep(2)
    screen = screen_cap()

    #new round
    play_button = cv2.imread('assets/play_button.png')
    result = cv2.matchTemplate(screen, play_button, cv2.TM_CCOEFF_NORMED)
    location = np.where(result == 1)
    newround = len(location[0]) > 0

    #game over
    restart_button = cv2.imread('assets/restart_button.png')
    result = cv2.matchTemplate(screen, restart_button, cv2.TM_CCOEFF_NORMED)
    location = np.where(result == 1)
    gameover = len(location[0]) > 0
    

    if gameover:
        #click restart
        pyautogui.moveTo(location[0][0], location[1][0])
        pydirectinput.click()

        time.sleep(.25)
        restart_text = cv2.imread('assets/restart_text.png')
        result = cv2.matchTemplate(screen_cap(), restart_text, cv2.TM_CCOEFF_NORMED)
        location = np.where(result >= .8)

        #confirm restart
        pyautogui.moveTo(location[1][0], location[0][0])
        pydirectinput.click()        
        return(2)
            
    if newround:
        return(1)
    else:
        return(0)

In [59]:

def define_grid(precision = 100, save = False):
    #get/crop image
    pyautogui.hotkey('alt', 'prtscr')
    img = ImageGrab.grabclipboard()
    img.save('temp.png')
    im = Image.open('temp.png')
    width, height = im.size

    left = 0 * width
    right = .85 * width
    top = .11 * height
    bottom = height

    image = im.crop((left, top, right, bottom))
    image_arr = np.array(image)

    #add grid
    fig, ax = plt.subplots()
    ax.imshow(image)
    ax.grid(True, color='white', linestyle='-')
    ax.set_xticks(np.arange(0, image_arr.shape[1], precision))
    ax.set_yticks(np.arange(0, image_arr.shape[0], precision))

    #get coordinates map
    x_centers = np.arange(precision / 2, image_arr.shape[1], precision)
    y_centers = np.arange(precision / 2, image_arr.shape[0], precision)
    X, Y = np.meshgrid(x_centers, y_centers)
    coordinates = np.column_stack((X.ravel(), Y.ravel()))

    if save:
        plt.savefig('map_grid.png')

    plt.close(fig)
    return(coordinates)

In [61]:
# Cost Data
import pandas as pd
import math

def get_costs(difficulty):
    base_costs = pd.read_csv('assets/base_costs.csv')
    upgrade_costs = pd.read_csv('assets/upgrade_costs.csv')

    if difficulty == 'easy':
        base_costs['cost'] * 0.85
        upgrade_costs['cost'] = upgrade_costs['cost'] * 0.85
        return(base_costs,upgrade_costs)
    
    if difficulty == 'hard':
        base_costs['cost'] = base_costs['cost'] * 1.08
        upgrade_costs['cost'] = upgrade_costs['cost'] * 1.08
        return(base_costs,upgrade_costs)
    
    if difficulty == 'impoppable':
        base_costs['cost'] = base_costs['cost'] * 1.2
        upgrade_costs['cost'] = upgrade_costs['cost'] * 1.2
        return(base_costs,upgrade_costs)
    
    else:
        return(base_costs,upgrade_costs)

    
base, upgrade = get_costs('hard')


In [31]:
#get health, money, round
from PIL import ImageGrab, Image, ImageDraw

def get_game_info():
    screen_cap()
    im = Image.open('temp.png')
    width, height = im.size

    ##### HP + Money #####
    left = .07 * width
    right = .30 * width
    top = .048 * height
    bottom = .1 * height
    im1 = im.crop((left, top, right, bottom))
    im1.save('temp1.png')

    #obscure image
    image = Image.open("temp1.png").convert("L")
    draw = ImageDraw.Draw(image)
    box_coordinates = (90, 0, 231, 70)
    draw.rectangle(box_coordinates, fill="black")
    image = image.point(lambda p: 255 if p > 250 else 0)
    image.save("temp1.png")

    hp, money = bfn.img_to_num('temp1.png') #remove bfn

    #### Round #####
    left = .745 * width
    right = .7728 * width
    top = .048 * height
    bottom = .1 * height
    im2 = im.crop((left, top, right, bottom))
    im2.save('temp2.png')

    #obscure image
    image = Image.open("temp2.png").convert("L")
    image = image.point(lambda p: 255 if p > 250 else 0)
    image.save("temp2.png")

    round = bfn.img_to_num('temp2.png') #REMOVE BFN

    return(hp,money,round)


In [62]:
time.sleep(1)
coords = pd.DataFrame(define_grid())
base_costs, upgrade_costs = get_costs('easy')
#place_tower(tower)


In [86]:
# check money
towers_list = []



money = 500
round = 5




# filter to afforded costs
towers_afforded = base_costs[base_costs['cost'] <= money]

# select tower
tower = towers_afforded.sample(n=1)
tower = tower.reset_index(drop=True)
name = tower['tower'].iloc[0]
cost = float(tower['cost'].iloc[0])

# select space and drop from coords
space = coords.sample(n=1)
coords = coords.drop(space.index).reset_index(drop=True)
space = space.reset_index(drop=True)
x=float(space[0].iloc[0])
y=float(space[1].iloc[0])

placement_list = [x, y, name, cost, round]
towers_list.append(placement_list)

# combine tower and space and index into new df
# obs = pd.concat([space,tower], axis=1)
# obs = obs.rename({0:'x',1:'y'}, axis=1)

#place tower
# pyautogui.moveTo(obs['x'], obs['y'])
# pydirectinput.press(obs['hotkey'])
# pydirectinput.click()        



In [89]:


def place_tower(base_costs,coords, round):
    towers_afforded = base_costs[base_costs['cost'] <= money]

    # select tower
    tower = towers_afforded.sample(n=1)
    tower = tower.reset_index(drop=True)
    name = tower['tower'].iloc[0]
    hotkey = tower['hotkey'].iloc[0]
    cost = float(tower['cost'].iloc[0])

    # select space and drop from coords
    space = coords.sample(n=1)
    coords = coords.drop(space.index).reset_index(drop=True)
    space = space.reset_index(drop=True)
    x=float(space[0].iloc[0])
    y=float(space[1].iloc[0])

    #ensure valid placement loop here
    pyautogui.moveTo(x, y)
    pydirectinput.press(hotkey)
    pydirectinput.click()

    #check if works first.
    placement_list = [x, y, name, cost, round]
    towers_list.append(placement_list)
    return


print(towers_list)

[[350.0, 950.0, 'tack_shooter', 260.0, 5], [1150.0, 950.0, 'wizard_monkey', 325.0, 5]]
[[350.0, 950.0, 'tack_shooter', 260.0, 5], [1150.0, 950.0, 'wizard_monkey', 325.0, 5], [1250.0, 650.0, 'ice_monkey', 500.0, 5]]


In [92]:
bfn.img_to_num('temp1.png')

[180, 95]

Tower Placement

1. Check money

2. choose random spot

3. place tower

4. check money again

4. if same as before, run again (print invalid placement)

5. maybe +- 5 to the coords a few times before choosing new coords


In [28]:
def img_to_num(img):   
    imgstr = pytesseract.image_to_string(img, lang='bloons')
    imgstr = imgstr.replace(',', '')
    num_list = [int(s) for s in re.findall(r'\d+', imgstr)]
    return(num_list)

#img_to_num('temp2.png')


pytesseract.image_to_string('temp2.png', lang='bloons')

''

In [93]:
time.sleep(1)
bfn.get_game_info()

(180, 95, [5])

In [11]:
import random

def spend(money,lives,last_money,last_lives): #manually tweak these values to find best performance
    result = False
    lives_probs = 0
    money_probs = 0
    random_chance = 20

    if(lives != last_lives):
        lives_lost = last_lives - lives
        lives_probs = 10 * lives_lost
    
    if(money > last_money):
        round_money = last_money - money
        money_probs = round_money/50

    totprobs = lives_probs + money_probs + random_chance
    selection = random.randint(1, 100)

    print(totprobs, selection)

    if(selection <= totprobs):
        result = True

    return(result)


spend(1000, 200, 950, 200)

19.0 42


False

In [219]:
import pandas as pd
towers_df = pd.read_csv('data.csv')
money = 2500
upgrade_costs = pd.read_csv('assets/upgrade_costs.csv')
print(towers_df)

       x      y           type  top  mid  bot
0  350.0  250.0  sniper_monkey    0    0    0
1  750.0  150.0  sniper_monkey    0    0    0


In [224]:
import pyautogui

i=0
while True:
    selected_tower = towers_df.sample(1)

    #check for crosspath
    upgraded_paths = int(sum([(selected_tower['top'].iloc[0] != 0),(selected_tower['mid'].iloc[0] != 0),(selected_tower['bot'].iloc[0] != 0)]))
    if upgraded_paths == 2:
        locked_path = selected_tower.columns[selected_tower.eq(0).any()][0]
        if locked_path == 'top':
            locked_path = 'top_path'
        if locked_path == 'mid':
            locked_path = 'middle_path'
        if locked_path == 'bot':
            locked_path = 'bottom_path'
    
    available_upgrades = upgrade_costs[(upgrade_costs['tower'].isin(selected_tower['type']))&
                                       (upgrade_costs['cost']<money)&
                                       (upgrade_costs['path'] != locked_path)]

    available_upgrades = available_upgrades[((available_upgrades['path']==('top_path'))&
                                            (available_upgrades['tier']==selected_tower['top'].iloc[0]+1))|
                                            ((available_upgrades['path']==('middle_path'))&
                                            (available_upgrades['tier']==selected_tower['mid'].iloc[0]+1))|
                                            ((available_upgrades['path']==('bottom_path'))&
                                            (available_upgrades['tier']==selected_tower['bot'].iloc[0]+1))]

    print(available_upgrades)

    if len(available_upgrades) != 0:
        selected_upgrade = available_upgrades.sample(1)
        break

    i=i+1
    if i == 5:
        print('No upgrades found.')
        break


x=float(selected_tower['x'].iloc[0])
y=float(selected_tower['y'].iloc[0])
pyautogui.moveTo(x, y)
pydirectinput.click()

selected_upgrade_path = selected_upgrade['path'].iloc[0]

if selected_upgrade_path == 'top_path':
    pydirectinput.press(',')
    selected_tower['top'] = selected_tower['top'] + 1
    towers_df.iloc[selected_tower.index] = selected_tower.iloc[0]

if selected_upgrade_path == 'middle_path':
    pydirectinput.press('.')
    selected_tower['mid'] = selected_tower['mid'] + 1
    towers_df.iloc[selected_tower.index] = selected_tower.iloc[0]

if selected_upgrade_path == 'bottom_path':
    #pydirectinput.press('/')
    selected_tower['bot'] = selected_tower['bot'] + 1
    towers_df.iloc[selected_tower.index] = selected_tower.iloc[0]

print(f"{selected_tower['type'].iloc[0]} upgraded to ({selected_tower['top'].iloc[0]},{selected_tower['mid'].iloc[0]},{selected_tower['bot'].iloc[0]})")
print(towers_df)

             tower         path  tier  cost
90   sniper_monkey     top_path     1   350
95   sniper_monkey  middle_path     1   250
100  sniper_monkey  bottom_path     1   450
sniper_monkey upgraded to (0,1,0)
       x      y           type  top  mid  bot
0  350.0  250.0  sniper_monkey    0    1    0
1  750.0  150.0  sniper_monkey    0    2    2


       x      y           type  top  mid  bot
0  350.0  250.0  sniper_monkey    1    0    1
2


SyntaxError: invalid syntax (1018104999.py, line 2)